In [ ]:
import copy
import json
import os.path as osp
import random
import re
from csv import DictReader
from district import District
from crop import Crop
from allocation import Allocation
import os
import argparse
import readfile
import numpy as np
import time
from geopy.distance import great_circle
from gurobipy import *

In [58]:
model = Model(name='CROP')
#3 dist, 2 crops from test 1
no_dist = 3
no_crop = 2
allocation = model.addVars(no_dist,no_crop,vtype=GRB.CONTINUOUS,lb = 0, name = 'allocation')  #For allocation
x = model.addVars(no_dist,no_crop*no_dist,vtype=GRB.CONTINUOUS,lb = 0, name = 'x')  #For transportation 

In [59]:
new_alloc = [[5,5],[5,5],[0,0]]
max_alloc = np.sum(new_alloc,axis=1)
print(max_alloc)

[10 10  0]


In [60]:
# for i in range(len(max_alloc)):
#     model.addConstr(quicksum(allocation[i,j] for j in range(no_crop))<=max_alloc[i])

In [61]:
cost = [[8000,10000],[5000,10000],[8000,10000]] #Cost per crop per ton
cost = np.array(cost)
crop_yield = [[1,2.5],[1,2],[0,0]]
crop_yield  = np.array(crop_yield)
price = [[1500,1100],[1500,1200],[1000,1000]]
price = np.array(price)
LD = [[4,10],[4,5],[2,0.7]]
LD = np.array(LD)
UD = [[10,15],[7,8],[4,1]]
UD = np.array(UD)
TC = [100,200]
TC = np.array(TC)
model.addConstrs((allocation[i,j]*crop_yield[i,j]-sum(x[i,j*no_dist+k] for k in range(no_dist))+sum(x[k,j*no_dist+i] for k in range(no_dist)))>=LD[i,j] for i in range(no_dist) for j in range(no_crop))
model.addConstrs((allocation[i,j]*crop_yield[i,j]-sum(x[i,j*no_dist+k] for k in range(no_dist))+sum(x[k,j*no_dist+i] for k in range(no_dist)))<=UD[i,j] for i in range(no_dist) for j in range(no_crop))
model.addConstrs((sum(allocation[i,j] for j in range(no_crop))<=max_alloc[i]) for i in range(no_dist))
distances = [[10^7,111.19508372,157.24960341],[111.19508372,10^7,111.19508372],[157.24960341,111.19508372,10^7]]
distances = np.array(distances)
#----------------------Method1-------------------
COP = quicksum(allocation[i,j]*cost[i,j] for i in range(no_dist) for j in range(no_crop))
Revenue = quicksum((allocation[i,j]*crop_yield[i,j]-sum(x[i,j*no_dist+k] for k in range(no_dist))+sum(x[k,j*no_dist+i] for k in range(no_dist)))*10*price[i,j] for i in range(no_dist) for j in range(no_crop))
Transport_cost = quicksum((TC[j]*distances[i,k]*x[i,j*no_dist+k]) for k in range(no_dist) for i in range(no_dist) for j in range(no_crop))
obj_fn =  Revenue - COP -  Transport_cost
#----------------------Method2--------------------
# obj_fn = quicksum((allocation[i,j]*crop_yield[i,j]-sum(x[i,j*no_dist+k] for k in range(no_dist))+sum(x[k,j*no_dist+i] for k in range(no_dist)))*10*price[i,j]-allocation[i,j]*cost[i,j]-sum(TC[j]*distances[i,k]*x[i,j*no_dist+k] for k in range(no_dist)) for i in range(no_dist) for j in range(no_crop))
model.setObjective(obj_fn,GRB.MAXIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 24 columns and 62 nonzeros
Model fingerprint: 0xd3fc04a1
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+03, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 2e+01]
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolved: 8 rows, 22 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1500000e+05   7.450000e+00   0.000000e+00      0s
       5    1.9979367e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.997936715e+05


In [62]:
values = model.getAttr("X", model.getVars())
values = np.array(values)
alloc = values[0:no_dist*no_crop].reshape((no_dist,no_crop))
transport_qty = values[no_dist*no_crop:].reshape((no_dist,no_dist*no_crop))
print(f'Allocation: \n {alloc}')
print(f'Transported quantity: \n {transport_qty}')

Allocation: 
 [[4. 6.]
 [6. 4.]
 [0. 0.]]
Transported quantity: 
 [[0.  0.  0.  0.  0.  0. ]
 [0.  0.  2.  0.  0.  0.7]
 [0.  0.  0.  0.  0.  0. ]]


In [63]:
stock = np.zeros(alloc.shape)
for i in range(no_dist):
    for j in range(no_crop):
        stock[i,j] = alloc[i,j]*crop_yield[i,j]-sum(transport_qty[i,j*no_dist+k] for k in range(no_dist))+sum(transport_qty[k,j*no_dist+i] for k in range(no_dist))
print(stock)
        # allocation[i,j]*crop_yield[i,j]-sum(x[i,j*no_dist+k] for k in range(no_dist))+sum(x[k,j*no_dist+i] for k in range(no_dist)))>=LD[i,j] for i in range(no_dist) for j in range(no_crop))

[[ 4.  15. ]
 [ 4.   7.3]
 [ 2.   0.7]]


In [64]:
COP = sum(alloc[i,j]*cost[i,j] for i in range(no_dist) for j in range(no_crop))
print(COP)
Revenue = sum((alloc[i,j]*crop_yield[i,j]-sum(transport_qty[i,j*no_dist+k] for k in range(no_dist))+sum(transport_qty[k,j*no_dist+i] for k in range(no_dist)))*10*price[i,j] for i in range(no_dist) for j in range(no_crop))
print(Revenue)
Transport_cost = sum((TC[j]*distances[i,k]*transport_qty[i,j*no_dist+k]) for k in range(no_dist) for i in range(no_dist) for j in range(no_crop))
print(Transport_cost)
print(Revenue - COP -  Transport_cost)

162000.0
399600.0
37806.3284648
199793.6715352


In [65]:
# for v in model.getVars():
#     print(f"{v.VarName} = {v.X}")

allocation[0,0] = 4.0
allocation[0,1] = 6.0
allocation[1,0] = 6.0
allocation[1,1] = 4.0
allocation[2,0] = 0.0
allocation[2,1] = 0.0
x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[0,4] = 0.0
x[0,5] = 0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 2.0
x[1,3] = 0.0
x[1,4] = 0.0
x[1,5] = 0.7
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 0.0
x[2,5] = 0.0


In [68]:
u = [[1,2,3],[4,5,6]]
v = [[1,2,3],[4,5,6]]
w= np.add(u,v)
print(w)

[[ 2  4  6]
 [ 8 10 12]]
